In [1]:
import torch
from torchvision import datasets

In [14]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from utils import labels2one_hot
from generate_data import generate_data, generate_grid_data
from modules import Dropout
from modules import Linear, Sequential
from modules.losses import LossMSE, LossCrossEntropy
from optimizers import SGD

In [4]:
def convert_to_one_hot_labels(input, target, val=0):
    tmp = input.new(target.size(0), target.max() + 1).fill_(-1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    if val == 0:      
        ret = (tmp+1)/2
    if val == -1:
        ret = tmp
    return ret

def load_data(one_hot_labels = False, normalize = False, flatten = True, data_dir = None, cifar = False, full = True, tiny = False, val = 0):

    if data_dir is None:
        """
        data_dir = os.environ.get('PYTORCH_DATA_DIR')
        if data_dir is None:
            data_dir = './data'
        """
        data_dir = './data'
        
    if cifar is not None and cifar:
        print('* Using CIFAR')
        cifar_train_set = datasets.CIFAR10(data_dir + '/cifar10/', train = True, download = True)
        cifar_test_set = datasets.CIFAR10(data_dir + '/cifar10/', train = False, download = True)

        train_input = torch.from_numpy(cifar_train_set.train_data)
        # Dirty hack to handle the change between torchvision 1.0.6 and 1.0.8
        if train_input.size(3) == 3:
            train_input = train_input.transpose(3, 1).transpose(2, 3).float()
        else:
            train_input = train_input.float()
        train_target = torch.LongTensor(cifar_train_set.train_labels)

        test_input = torch.from_numpy(cifar_test_set.test_data).float()
        # Dirty hack to handle the change between torchvision 1.0.6 and 1.0.8
        if test_input.size(3) == 3:
            test_input = test_input.transpose(3, 1).transpose(2, 3).float()
        else:
            test_input = test_input.float()
        test_target = torch.LongTensor(cifar_test_set.test_labels)

    else:
        print('* Using MNIST')
        mnist_train_set = datasets.MNIST(data_dir + '/mnist/', train = True, download = True)
        mnist_test_set = datasets.MNIST(data_dir + '/mnist/', train = False, download = True)

        train_input = mnist_train_set.train_data.view(-1, 1, 28, 28).float()
        train_target = mnist_train_set.train_labels
        test_input = mnist_test_set.test_data.view(-1, 1, 28, 28).float()
        test_target = mnist_test_set.test_labels

    if flatten:
        train_input = train_input.clone().view(train_input.size(0), -1)
        test_input = test_input.clone().view(test_input.size(0), -1)

    if full:
        if tiny:
            raise ValueError('Cannot have both --full and --tiny')
    else:
        if tiny:
            print('** Reduce the data-set to the tiny setup')
            train_input = train_input.narrow(0, 0, 500)
            train_target = train_target.narrow(0, 0, 500)
            test_input = test_input.narrow(0, 0, 100)
            test_target = test_target.narrow(0, 0, 100)
        else:
            print('** Reduce the data-set (use --full for the full thing)')
            train_input = train_input.narrow(0, 0, 1000)
            train_target = train_target.narrow(0, 0, 1000)
            test_input = test_input.narrow(0, 0, 1000)
            test_target = test_target.narrow(0, 0, 1000)

    print('** Use {:d} train and {:d} test samples'.format(train_input.size(0), test_input.size(0)))

    if one_hot_labels:
        train_target = convert_to_one_hot_labels(train_input, train_target, val=val)
        test_target = convert_to_one_hot_labels(test_input, test_target, val=val)

    if normalize:
        mu, std = train_input.mean(), train_input.std()
        train_input.sub_(mu).div_(std)
        test_input.sub_(mu).div_(std)

    return train_input, train_target, test_input, test_target


In [5]:
train_input, train_target, test_input, test_target = load_data(one_hot_labels = True, normalize = True, flatten = True, data_dir = None, cifar = False, full = True, tiny = False, val=0)
train_input_mse, train_target_mse, test_input_mse, test_target_mse = load_data(one_hot_labels = True, normalize = True, flatten = True, data_dir = None, cifar = False, full = True, tiny = False, val=-1)

* Using MNIST
** Use 60000 train and 10000 test samples
* Using MNIST
** Use 60000 train and 10000 test samples


In [6]:
train_target

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.]])

In [7]:
train_target_mse

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1.,  1., -1.]])

In [10]:
def default_net_1(x_all, y_all, num_of_neurons=(2, 25, 25, 25, 2), lr=0.1, momentum_coef=0.0, weight_decay=0.0, num_of_epochs=100):
    ce = LossCrossEntropy()

    model = Sequential()
    model.add(Linear(out=num_of_neurons[1], input_size=num_of_neurons[0], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[2], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[3], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[4], activation='softmax'))

    model.loss = ce
    sgd = SGD(lr, momentum_coef, weight_decay=0.0)

    sgd.train(model, x_all, y_all, num_of_epochs, val_split=0.2, verbose=0)

    return model

def default_net_2(x_all, y_all, num_of_neurons=(2, 25, 25, 25, 2), lr=0.1, momentum_coef=0.0, weight_decay=0.0, num_of_epochs=100):
    ce = LossCrossEntropy()

    model = Sequential()
    model.add(Linear(out=num_of_neurons[1], input_size=num_of_neurons[0], activation='relu'))
    model.add(Linear(out=num_of_neurons[2], activation='relu'))
    model.add(Linear(out=num_of_neurons[3], activation='relu'))
    model.add(Linear(out=num_of_neurons[4], activation='softmax'))

    model.loss = ce
    sgd = SGD(lr, momentum_coef, weight_decay=0.2)

    sgd.train(model, x_all, y_all, num_of_epochs, val_split=0.2, verbose=0)

    return model

def default_net_3(x_all, y_all, num_of_neurons=(2, 25, 25, 25, 2), lr=0.1, momentum_coef=0.0, num_of_epochs=100):
    mse = LossMSE()

    model = Sequential()
    model.add(Linear(out=num_of_neurons[1], input_size=num_of_neurons[0], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[2], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[3], activation='relu'))
    model.add(Dropout(prob=0.2))
    model.add(Linear(out=num_of_neurons[4], activation='relu'))

    model.loss = mse
    sgd = SGD(lr, momentum_coef, weight_decay=0.2)

    sgd.train(model, x_all, y_all, num_of_epochs, val_split=0.2, verbose=0)

    return model

In [11]:
model1 = default_net_1(train_input, train_target, num_of_neurons=(784, 100, 100, 100, 10), lr=0.1, momentum_coef=0.0, num_of_epochs=100)
model1.evaluate(test_input, test_target, return_pred=True)

prev: 100, current: 784
Added Module Name: 0_Linear 
Added Module Name: 1_ReLU 
Added Module Name: 2_Dropout 
prev: 100, current: 100
Added Module Name: 3_Linear 
Added Module Name: 4_ReLU 
Added Module Name: 5_Dropout 
prev: 100, current: 100
Added Module Name: 6_Linear 
Added Module Name: 7_ReLU 
Added Module Name: 8_Dropout 
prev: 10, current: 100
Added Module Name: 9_Linear 
Added Module Name: 10_Softmax 


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\furkan\Desktop\EPFL\ma\ma2\Deep Learning\project\project2\dl-framework-master\dl-framework-master\modules\losses.py:94: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return loss_val[0]  # TODO: handle this accordingly with take_avg false
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:07<00:00,  1.88s/it]


(tensor(0.8321), tensor(0.5442), tensor([ 7,  2,  1,  ...,  4,  8,  6]))

In [12]:
model2 = default_net_2(train_input, train_target, num_of_neurons=(784, 100, 100, 100, 10), lr=0.1, momentum_coef=0.0, num_of_epochs=100)
model2.evaluate(test_input, test_target, return_pred=True)

prev: 100, current: 784
Added Module Name: 0_Linear 
Added Module Name: 1_ReLU 
prev: 100, current: 100
Added Module Name: 2_Linear 
Added Module Name: 3_ReLU 
prev: 100, current: 100
Added Module Name: 4_Linear 
Added Module Name: 5_ReLU 
prev: 10, current: 100
Added Module Name: 6_Linear 
Added Module Name: 7_Softmax 


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\furkan\Desktop\EPFL\ma\ma2\Deep Learning\project\project2\dl-framework-master\dl-framework-master\modules\losses.py:94: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return loss_val[0]  # TODO: handle this accordingly with take_avg false
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:47<00:00,  1.07s/it]


(tensor(0.9052), tensor(0.3296), tensor([ 7,  2,  1,  ...,  4,  5,  6]))

In [15]:
model3 = default_net_3(train_input_mse, train_target_mse, num_of_neurons=(784, 100, 100, 100, 10), lr=0.1, momentum_coef=0.0, num_of_epochs=100)
model3.evaluate(test_input_mse, test_target_mse, return_pred=True)

prev: 100, current: 784
Added Module Name: 0_Linear 
Added Module Name: 1_ReLU 
Added Module Name: 2_Dropout 
prev: 100, current: 100
Added Module Name: 3_Linear 
Added Module Name: 4_ReLU 
Added Module Name: 5_Dropout 
prev: 100, current: 100
Added Module Name: 6_Linear 
Added Module Name: 7_ReLU 
Added Module Name: 8_Dropout 
prev: 10, current: 100
Added Module Name: 9_Linear 
Added Module Name: 10_ReLU 


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:23<00:00,  2.04s/it]


(tensor(1.00000e-02 *
        9.8200), tensor(10.), tensor([ 0,  0,  0,  ...,  0,  0,  0]))